# Jax's Loops

## Lesson Goals:

By the end of this lesson, you'll be able to articulate why and when you want to use `jax`'s native `while_loop`, `fori_loop`, and `scan` over python's native loops. In the process, you'll learn how to read haskell-like type signatures, which will be useful as you explore the `jax` library further.

## Core Concepts:

- functional programming 
- reading Haskell function signatures
- `while_loop`
- `fori_loop`
- `scan`

## Concepts In action:

- Easy: [lotka-volterra](../case_studies/lotka-volterra/README.md)

- Intermediate: [leaky_integrate_and_fire](../case_studies/leaky_integrate_and_fire/README.md)

In [ ]:
import numpy as np
import jax.numpy as jnp
import jax

np.random.seed(42)

# Haskell-like signatures

![](../assets/haskell.png)

type signatures are a great way to abstractly understand functions and what they do. Let's walk through a few examples:

## Signature 1: Mapping functions over elements of an iterable

```haskell
map :: (a -> b) -> [a] -> [b]
```

```python
def map(func: Callable, arr):
    return ...
```

## Signature 2: Joining Structures

```haskell
(++) :: [a] -> [a] -> [a]
```

What sort of pre-condition

```python
def abstract_join_of_iterable(s1, s2):
    """
    TODO: answer the following
    1) what sort of pre-condition do we need? What function must type(s1) implement?
    2) assuming the pre-condition, what might this function look like?
    """
    return ...
```


## Signature 3: Filtering elements

```haskell
filter :: (a -> Bool) -> [a] -> [a]
```

```python
def filter(filter_func, iterable):
    """
    TODO: answer the following
    1) must it be a list? 
    2) implement the function
    """

```

# Looping in Jax

As mentioned before, you probably don't want to `jit` a function that has a native python `for-loop` in it as this increases your compilation time. Thankfully, `jax` provides:

- [jax.lax.while_loop](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.while_loop.html#jax.lax.while_loop)
- [jax.lax.fori_loop](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.fori_loop.html)
- [jax.lax.scan](https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.scan.html)

to circumvent this issue. 

Note: we don't necessarily see a speedup in runtime; the primary advantage of using these jax functions is that the compilation time is reduced.


## The Jax Functions we will be covering:

```haskell
while_loop :: (a -> Bool) -> (a -> a) -> a -> a
```

```haskell
fori_loop :: Int -> Int -> ((Int, a) -> a) -> a -> a
```

```haskell
scan :: (c -> a -> (c, b)) -> c -> [a] -> (c, [b])
```

## Jax's Loops

What their corresponding implementations might look like in Python

```python
def while_loop(cond_fun, body_fun, init_val):
    val = init_val
    while cond_fun(val):
        val = body_fun(val)
    return val
```

```python
def fori_loop(lower, upper, body_fun, init_val):
    val = init_val
    for i in range(lower, upper):
        val = body_fun(i, val)
    return val
```


```python
def scan(f, init, xs):
    carry = init
    ys = []
    for x in xs:
        carry, y = f(carry, x)
        ys.append(y)
  return carry, np.stack(ys)
```


# Exercises

While answering the questions, try to only refer to the following signatures. If you absolutely must, you may refer to the python "implementation" above

```haskell
while_loop :: (a -> Bool) -> (a -> a) -> a -> a
```

```haskell
fori_loop :: Int -> Int -> ((Int, a) -> a) -> a -> a
```

```haskell
scan :: (c -> a -> (c, b)) -> c -> [a] -> (c, [b])
```



In [ ]:
def while_loop_exercise():
    """
    TODO: 
        implement a function that, given an array of initial values, squares it. While it's greater than
        the TOL, keep going.
        
        If implemented correctly, you should see only a few elements less than 1e-16
    Returns:

    """
    TOL = 1e-16
    initial_values = jnp.asarray(np.random.uniform(low=0.5, high=1, size=(5, 5)))
    print("Starting values:")
    print(initial_values)
    
    def cond(v):
        return jnp.all(v > TOL)
   
    def body_fun(v):
        return v ** 2
        
    
    resp = jax.lax.while_loop(cond, body_fun, initial_values) 
    print("Ending Values")
    print(resp)
    assert jnp.any(resp < TOL)
    print("While loop exercise passed!")
    
def scan_exercise():
    """
    TODO:
        implement a function that, given a 2xN matrix, implements a scan such that the first vector is a sum of the initial values,
            and the second vector is the product of the initial vector
    Returns:

    """
    vector = jnp.asarray([
        [1, 2, 3, 4, 5],  # Final result should be 15
        [2, 4, 6, 8, 10]  # Final result should be 3840
    ]).T
    
    def f(carry, x):
        v1 = carry[0] + x[0]
        v2 = carry[1] * x[1]
        new_carry = jnp.asarray([v1, v2])
        
        return new_carry, new_carry
        
    
    final_result, intermediate_results = jax.lax.scan(f, init=jnp.asarray([0, 1]), xs=vector)
    assert jnp.all(final_result == jnp.asarray([15, 3840]))
    assert jnp.all(intermediate_results == jnp.asarray(
        [
            [1, 3, 6, 10, 15], 
            [2, 8, 48, 384, 3840]
        ]).T)
    
    print("Scan Exercise passed!")
    
    
while_loop_exercise()
print("\n")
scan_exercise()

# Further Exercises: 

0) Read through [case_studies/leaky_integrate_and_fire/jax_leaky_integrate_and_fire_3_scan.ipynb](../case_studies/leaky_integrate_and_fire/jax_leaky_integrate_and_fire_3_scan.ipynb) for an example of using `scan` in a "real-world" scenario

1) Read through [case_studies/lotka-volterra/jax_lotka-volterra.ipynb](../case_studies/lotka-volterra/jax_lotka-volterra.ipynb) which shows you the various ways you can use a `fori_loop` and `scan` 

